In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrindex.index_exo_base import IndexEXOBase
from datetime import datetime

In [ ]:
from bdateutil import relativedelta
from tmqr.logs import log
from tmqrfeed.quotes import QuoteContFut
from tmqrfeed import Costs
import pandas as pd

class EXOSpreadIndex(IndexEXOBase):
    _description_short = "EXO Vanilla Long/Short spread index"
    _description_long = ""
    _index_name = "EXOSpreadFixed"
    
    def __init__(self, datamanager, **kwargs):
        super().__init__(datamanager, **kwargs)
        
        self.PRIMARY_INSTRUMENT = 'US.ES'
        self.SECONDARY_INSTRUMENT = 'US.CL'
              
        self.costs_futures = 3.0
        self.costs_options = 3.0
    
    def setup(self):
        
        #
        # IMPORTANT! Use trading session of self.PRIMARY_INSTRUMENT 
        #   All US.CL quotes and positions will use 'US.ES' decision and execution time
        # 
        self.dm.session_set(self.PRIMARY_INSTRUMENT, session_instance=self.session)
        
        #
        # Set primary quotes for 'US.ES' to align all data to its index
        #
        self.dm.series_primary_set(QuoteContFut, self.PRIMARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        
        self.dm.series_extra_set(self.SECONDARY_INSTRUMENT, QuoteContFut, self.SECONDARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        #
        # Set index costs (costs are calculated at the final stage, of index equirt calculation)
        # 
        self.dm.costs_set(self.PRIMARY_INSTRUMENT.split('.')[0], Costs(per_contract=self.costs_futures,
                                                               per_option=self.costs_options))
        
    
    def calc_exo_logic(self):
        """
        Calculates SmartEXO logic.
        NOTE: this method must use self.dm.quotes() or self.dm.quotes(series_key='for_secondary_series') to 
              calculate SmartEXO logic
        :return: Pandas.DataFrame with index like in dm.quotes() (i.e. primary quotes)
        """
        # Getting quotes
        primary_quotes = self.dm.quotes()
        secondary_quotes = self.dm.quotes(self.SECONDARY_INSTRUMENT)
        
        # Getting instrument information 
        primary_instrument_info = self.dm.instrument_info_get(self.PRIMARY_INSTRUMENT)
        secondary_instrument_info = self.dm.instrument_info_get(self.SECONDARY_INSTRUMENT)
        
        # Calculating point value
        primary_instrument_point_value = 1.0 / primary_instrument_info.ticksize * primary_instrument_info.tickvalue
        secondary_instrument_point_value = 1.0 / secondary_instrument_info.ticksize * secondary_instrument_info.tickvalue
        
        # Calculating USD value price series
        primary_usd_value = primary_quotes['c'] * primary_instrument_point_value
        secondary_usd_value = secondary_quotes['c'] * secondary_instrument_point_value
        
        # Calculating USD value ratio per 10 contracts        
        usd_ratio = primary_usd_value / secondary_usd_value
        
        # Add extra logic to illustrate how SmartEXO can be implemeted
        # Define bull trend regime as close of primary > moving_average(primary, 20-periods)
        primary_in_bull_trend = primary_quotes['c'] > primary_quotes['c'].rolling(20).mean()
        
        # We have to return pandas.DataFrame class
        return pd.DataFrame({
            'usd_ratio': usd_ratio,
            # Optionally include spread prices
            'primary_usd_value': primary_usd_value,
            'secondary_usd_value': secondary_usd_value,
            # Include SMART EXO regime
            'primary_in_bull_trend': primary_in_bull_trend,
        })
    
    def manage_position(self, dt, pos, logic_df):
        """
        Manages opened position (rollover checks/closing, delta hedging, etc)
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """        
        #
        # Check expiration moment
        # Or you can check custom days to expiration values
        #  pos.almost_expired_ratio(dt, rollover_days_before_fut=5, rollover_days_before_opt=7)
        if pos.almost_expired_ratio(dt) > 0:                        
            pos.close(dt)
            
        #
        # Check business days after last transaction
        #
        pos_last_transaction_date = pos.last_transaction_date(dt)        
        log.debug("Last transaction date: {0}".format(pos_last_transaction_date))
        days_after_last_trans = relativedelta(dt, pos_last_transaction_date).bdays
        
        if days_after_last_trans > 7:
            log.debug("Business days > 7, closing position")
            # Close the position
            pos.close(dt)
            # Avoid following checks            
            return 
        
        #
        # Delta based rebalance
        #
        delta = pos.delta(dt)
        if delta > 0.7:
            log.debug("Delta > 0.7")
            # Close the position
            pos.close(dt)
            # Avoid following checks            
            return 
                
        #
        # logic_df based rebalance
        #
        primary_in_bull_trend = logic_df['primary_in_bull_trend']
        
        if not primary_in_bull_trend: 
            log.debug("not primary_in_bull_trend")
            # Close the position
            pos.close(dt)
            # Avoid following checks            
            return 

    def construct_position(self, dt, pos, logic_df):
        """
        EXO position construction method
        
        NOTE!: this method only called when there is no active position for 'dt'
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """
        # Getting active futures and options chains
        fut_primary, opt_chain_primary = self.dm.chains_options_get(self.PRIMARY_INSTRUMENT, dt)
        fut_secondary, opt_chain_secondary = self.dm.chains_options_get(self.SECONDARY_INSTRUMENT, dt)
        
        # Getting logic_df slice information
        # Example of slice data:
        """
        > print(logic_df)
        primary_in_bull_trend      False
        primary_usd_value         103788
        secondary_usd_value        44880
        usd_ratio                2.31256
        Name: 2016-05-02 12:40:00-07:00, dtype: object        
        
        > print(logic_df['usd_ratio'])
        2.31256
        
        > print(logic_df['primary_usd_value'])
        103788        
        """
        # You can easily access data calculated in calc_exo_logic() metod by
        # logic_df['primary_usd_value']
        # logic_df['some_column_name']
        usd_ratio = logic_df['usd_ratio']
        primary_in_bull_trend = logic_df['primary_in_bull_trend']
        
        # PRIMARY long
        primary_qty = 5.0 
        pos.add_transaction(dt, fut_primary, primary_qty)
        # SECONDARY short
        secondary_qty = round(primary_qty * usd_ratio)
        pos.add_transaction(dt, fut_secondary, -secondary_qty)
        
        
        # Usign SmartEXO hedge style
        if primary_in_bull_trend:
            # Add primary long call if primary_in_bull_trend
            target_delta = 0.4
            hedge_qty = round(primary_qty * target_delta)
            
            pos.add_transaction(dt, opt_chain_primary.find(dt, target_delta, 'C', how='delta'), hedge_qty)
        

In [ ]:
dm = DataManager(date_start=datetime(2016, 5, 1))

In [ ]:
INDEX_CONTEXT = {
    'instrument': "US.ES",
    'costs_futures': 3.0,
    'costs_options': 3.0,
}
index = EXOSpreadIndex(dm, **INDEX_CONTEXT)

#
# BOTH index init code lines are equal
#

#index = EXODeltaTargetGeneric(dm, instrument="US.ES", costs_futures=3.0, costs_options=3.0)

In [ ]:
#index.setup()
#index.calc_exo_logic().tail()

In [ ]:
index.run()


## Index equity

In [ ]:
list(index.data.columns)

# Spread position

Both of ES and CL have the same decision time and price



In [ ]:
index.position

In [ ]:
index.data['equity_execution'].plot()
title(index.index_name)

In [ ]:
index.index_name